In [1]:
import os
import pdfplumber
import pandas as pd
import re
from bs4 import BeautifulSoup
import os
import pdfplumber
import pandas as pd
import re

## 1. Loading quarterly / annual financial reports files

In [2]:
tickers = ['ARCC', 'FSK', 'ORCC', 'HTGC', 'MAIN', 'TSLX', 'GBDC', 'PSEC', 'GSBD', 'NMFC',
          'OCSL', 'AINV', 'TCPC', 'NEWT', 'BCSF', 'CGBD', 'SLRC', 'BBDC', 'CSWC', 'GAIN',
          'PFLT', 'TPVG', 'PNNT', 'FDUS', 'GLAD']

In [3]:
import glob

In [4]:
file_list = []
for ticker in tickers:
    for name in glob.glob('BDC_edgar/sec-edgar-filings/{}/10-*/*/filing-details.html'.format(ticker)):
        file_list.append(name)

In [5]:
file_list[:10]

['BDC_edgar/sec-edgar-filings/ARCC/10-Q/0001104659-05-055036/filing-details.html',
 'BDC_edgar/sec-edgar-filings/ARCC/10-Q/0001104659-08-050548/filing-details.html',
 'BDC_edgar/sec-edgar-filings/ARCC/10-Q/0001104659-09-047554/filing-details.html',
 'BDC_edgar/sec-edgar-filings/ARCC/10-Q/0001104659-12-073940/filing-details.html',
 'BDC_edgar/sec-edgar-filings/ARCC/10-Q/0001104659-07-080878/filing-details.html',
 'BDC_edgar/sec-edgar-filings/ARCC/10-Q/0001104659-15-055586/filing-details.html',
 'BDC_edgar/sec-edgar-filings/ARCC/10-Q/0001104659-13-037917/filing-details.html',
 'BDC_edgar/sec-edgar-filings/ARCC/10-Q/0001104659-16-136353/filing-details.html',
 'BDC_edgar/sec-edgar-filings/ARCC/10-Q/0001287750-21-000063/filing-details.html',
 'BDC_edgar/sec-edgar-filings/ARCC/10-Q/0001104659-07-037848/filing-details.html']

In [6]:
len(file_list)

1184

### There are 1,184 quarterly/annual financial reports of 25 BDC companies from 2001 to 2021

#### Let's load / clean all the text data in HTM files

In [7]:
def cleaning(text):
    # text = ''.join(map(str, ls_text))
    elements = text.split('\n')
    
    lines = []
    for sent in elements:
        sent = sent.replace('\xa0', ' ')
        lines.append(sent)

    while('' in lines):
        lines.remove('')
    while(' ' in lines):
        lines.remove(' ')
    while('  ' in lines):
        lines.remove('  ')
    while('   ' in lines):
        lines.remove('   ')
        
    finaltext = ' '.join(lines)
    
    return finaltext

In [8]:
def loadHTM(filelist):
    ls_text = []
    for i in range(len(filelist)):
        with open(filelist[i]) as fp:
            soup = BeautifulSoup(fp)
            text = soup.get_text()
        cleaned_text = cleaning(text)
        ls_text.append(cleaned_text)
        # ls_text.append(text) 
    return ls_text

In [9]:
def convert_df(ls1,ls2):
    df = pd.DataFrame(columns=['file_name', 'document_content'])
    for i in range(len(ls1)):
        df = df.append(pd.DataFrame({"file_name":ls2[i],"document_content":ls1[i]}, index = [0]), ignore_index = True)
    return df

In [10]:
def read_keyword(filepath):
    df1 = pd.read_excel(filepath)
    return df1

In [11]:
def extract_file(df1,df2):
    df3 = pd.DataFrame(columns=['file_name', 'key_words', 'related_statements'])
    for i in range(df1.shape[0]):
        #print(i)
        for word in df2['keyword']:
            #print(word)
            # result = re.findall('({}.*?)\.'.format(word), df1['document content'][i], re.S)
            result = re.findall(r"([^.]*?{}[^.]*\.)".format(word), df1['document_content'][i])
            for j in result:
                #print(j)
                df3 = df3.append(pd.DataFrame({"file_name":df1['file_name'][i], "key_words":word, "related_statements":j},index = [0]), ignore_index = True)
    # df3.to_excel(outpath, index=False)           
    return df3

In [12]:
ls_text = loadHTM(file_list)
df_pdf = convert_df(ls_text, file_list)

In [13]:
df_pdf

,file_name,document_content
0,BDC_edgar/sec-edgar-filings/ARCC/10-Q/00011046...,10-Q 1 a05-18287_110q.htm QUARTERLY REPORT PUR...
1,BDC_edgar/sec-edgar-filings/ARCC/10-Q/00011046...,10-Q 1 a08-18753_110q.htm 10-Q UNITED STATES S...
2,BDC_edgar/sec-edgar-filings/ARCC/10-Q/00011046...,10-Q 1 a09-18546_110q.htm 10-Q Table of Conten...
3,BDC_edgar/sec-edgar-filings/ARCC/10-Q/00011046...,10-Q 1 a12-20006_110q.htm 10-Q Table of Conten...
4,BDC_edgar/sec-edgar-filings/ARCC/10-Q/00011046...,10-Q 1 a07-25628_110q.htm 10-Q UNITED STATES S...
...,...,...
1179,BDC_edgar/sec-edgar-filings/GLAD/10-K/00011046...,10-K 1 a09-34045_110k.htm 10-K Table of Conten...
1180,BDC_edgar/sec-edgar-filings/GLAD/10-K/00011931...,10-K 1 d263060d10k.htm 10-K 10-K Table of Cont...
1181,BDC_edgar/sec-edgar-filings/GLAD/10-K/00011931...,10-K 1 d255684d10k.htm 10-K 10-K Table of Cont...
1182,BDC_edgar/sec-edgar-filings/GLAD/10-K/00011931...,10-K 1 d631347d10k.htm 10-K 10-K UNITED STAT...


In [14]:
df_pdf_sorted = df_pdf.sort_values(by = ['file_name'], ascending = True, ignore_index = True)
df_pdf_sorted

,file_name,document_content
0,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...
1,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...
2,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...
3,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...
4,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K FOR THE FISCAL YEAR ...
...,...,...
1179,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20190930.htm 10-Q tslx-10q_201...
1180,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200331.htm 10-Q tslx-10q_202...
1181,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200630.htm 10-Q tslx-10q_202...
1182,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200930.htm 10-Q tslx-10q_202...


In [15]:
df_pdf_sorted.head()

,file_name,document_content
0,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...
1,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...
2,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...
3,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...
4,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K FOR THE FISCAL YEAR ...


In [16]:
def findTicker(text):
    for ticker in tickers:
        if ticker in text:
            return ticker

In [17]:
forms = ['10-K', '10-Q']

def findForm(text):
    for form in forms:
        if form in text:
            return form

In [18]:
def findPeriod(text):
    return text[-29:-20]

In [19]:
df_pdf_sorted['ticker'] = df_pdf_sorted['file_name'].apply(findTicker)
df_pdf_sorted['file_type'] = df_pdf_sorted['file_name'].apply(findForm)
df_pdf_sorted['period'] = df_pdf_sorted['file_name'].apply(findPeriod)

In [20]:
df_pdf_sorted

,file_name,document_content,ticker,file_type,period
0,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...,AINV,10-K,05-125223
1,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...,AINV,10-K,07-124916
2,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...,AINV,10-K,08-123776
3,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...,AINV,10-K,09-122451
4,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K FOR THE FISCAL YEAR ...,AINV,10-K,10-129519
...,...,...,...,...,...
1179,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20190930.htm 10-Q tslx-10q_201...,TSLX,10-Q,19-040349
1180,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200331.htm 10-Q tslx-10q_202...,TSLX,10-Q,20-021239
1181,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200630.htm 10-Q tslx-10q_202...,TSLX,10-Q,20-036104
1182,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200930.htm 10-Q tslx-10q_202...,TSLX,10-Q,20-050467


In [21]:
finalDF = df_pdf_sorted.sort_values(by = ['ticker','period'], ascending = True, ignore_index = True)

In [22]:
finalDF

,file_name,document_content,ticker,file_type,period
0,BDC_edgar/sec-edgar-filings/AINV/10-Q/00009478...,10-Q 1 f10q_020605.htm QUARTERLY REPORT Quarte...,AINV,10-Q,05-000318
1,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...,AINV,10-K,05-125223
2,BDC_edgar/sec-edgar-filings/AINV/10-Q/00011931...,10-Q 1 d10q.htm FORM 10-Q Form 10-Q Table of C...,AINV,10-Q,05-156270
3,BDC_edgar/sec-edgar-filings/AINV/10-Q/00011931...,10-Q 1 d10q.htm FOR THE QUARTER ENDED SEPTEMBE...,AINV,10-Q,05-215629
4,BDC_edgar/sec-edgar-filings/AINV/10-Q/00011931...,10-Q 1 d10q.htm FORM 10-Q - FOR THE QUARTER EN...,AINV,10-Q,07-022555
...,...,...,...,...,...
1179,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200331.htm 10-Q tslx-10q_202...,TSLX,10-Q,20-021239
1180,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200630.htm 10-Q tslx-10q_202...,TSLX,10-Q,20-036104
1181,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200930.htm 10-Q tslx-10q_202...,TSLX,10-Q,20-050467
1182,BDC_edgar/sec-edgar-filings/TSLX/10-K/00015645...,10-K 1 tslx-10k_20201231.htm 10-K tslx-10k_202...,TSLX,10-K,21-006366


In [23]:
outpath = 'financial_reports.csv'
finalDF.to_csv(outpath, index=False)

## 2. Load stock and dividend history

In [25]:
!pip install yfinance

In [39]:
import yfinance as yf

In [80]:
print(tickers)

['ARCC', 'FSK', 'ORCC', 'HTGC', 'MAIN', 'TSLX', 'GBDC', 'PSEC', 'GSBD', 'NMFC', 'OCSL', 'AINV', 'TCPC', 'NEWT', 'BCSF', 'CGBD', 'SLRC', 'BBDC', 'CSWC', 'GAIN', 'PFLT', 'TPVG', 'PNNT', 'FDUS', 'GLAD']


### Load stock history

In [81]:
stock_history = yf.download(tickers,'1900-01-01','2021-08-01') 

[*********************100%***********************]  25 of 25 completed


In [82]:
stock_history

Adj Close                                                        \
                AINV       ARCC   BBDC   BCSF   CGBD       CSWC       FDUS   
Date                                                                         
1980-03-17       NaN        NaN    NaN    NaN    NaN   0.338482        NaN   
1980-03-18       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-19       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-20       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-25       NaN        NaN    NaN    NaN    NaN   0.312839        NaN   
...              ...        ...    ...    ...    ...        ...        ...   
2021-07-26     13.85  20.030001  10.53  15.30  13.45  25.389999  16.830000   
2021-07-27     13.81  20.120001  10.53  15.20  13.39  25.180000  16.799999   
2021-07-28     13.81  20.379999  10.52  15.22  13.35  25.180000  16.730000   
2021-07-29     13.73  19.850000  10.58  15.20  13.33  25.150000  16.760000   
2021-07-30     13.75  19.969999  10.72  15.11  13.25  24.870001  16.799999   

                                     ...    Volume                      \
              FSK       GAIN   GBDC  ...      NMFC      OCSL      ORCC   
Date                                 ...                                 
1980-03-17    NaN        NaN    NaN  ...       NaN       NaN       NaN   
1980-03-18    NaN        NaN    NaN  ...       NaN       NaN       NaN   
1980-03-19    NaN        NaN    NaN  ...       NaN       NaN       NaN   
1980-03-20    NaN        NaN    NaN  ...       NaN       NaN       NaN   
1980-03-25    NaN        NaN    NaN  ...       NaN       NaN       NaN   
...           ...        ...    ...  ...       ...       ...       ...   
2021-07-26  21.09  14.299312  15.94  ...  202400.0  423000.0  713200.0   
2021-07-27  21.00  14.199804  15.95  ...  281300.0  274900.0  806200.0   
2021-07-28  21.01  14.229656  16.00  ...  340200.0  525700.0  854500.0   
2021-07-29  21.02  14.259508  15.87  ...  258900.0  327400.0  538700.0   
2021-07-30  20.99  14.309262  15.78  ...  319400.0  443700.0  570200.0   

                                                                         \
                PFLT      PNNT       PSEC      SLRC      TCPC      TPVG   
Date                                                                      
1980-03-17       NaN       NaN        NaN       NaN       NaN       NaN   
1980-03-18       NaN       NaN        NaN       NaN       NaN       NaN   
1980-03-19       NaN       NaN        NaN       NaN       NaN       NaN   
1980-03-20       NaN       NaN        NaN       NaN       NaN       NaN   
1980-03-25       NaN       NaN        NaN       NaN       NaN       NaN   
...              ...       ...        ...       ...       ...       ...   
2021-07-26  104300.0  200500.0  1401600.0  122400.0  187100.0   43600.0   
2021-07-27   83900.0  194900.0  1531100.0   72600.0  108100.0   53100.0   
2021-07-28  105400.0  101100.0  1008100.0   61600.0  113900.0   86900.0   
2021-07-29  241900.0  199200.0  1019500.0   79600.0  153300.0   70700.0   
2021-07-30  181800.0  252900.0  1296100.0   64100.0  148100.0  114500.0   

                      
                TSLX  
Date                  
1980-03-17       NaN  
1980-03-18       NaN  
1980-03-19       NaN  
1980-03-20       NaN  
1980-03-25       NaN  
...              ...  
2021-07-26  182500.0  
2021-07-27  153300.0  
2021-07-28  168600.0  
2021-07-29  239100.0  
2021-07-30  253800.0  

[8848 rows x 150 columns]

In [83]:
stock_history.reset_index(level = 0, inplace = True)
stock_history

Date Adj Close                                             \
                     AINV       ARCC   BBDC   BCSF   CGBD       CSWC   
0    1980-03-17       NaN        NaN    NaN    NaN    NaN   0.338482   
1    1980-03-18       NaN        NaN    NaN    NaN    NaN   0.333353   
2    1980-03-19       NaN        NaN    NaN    NaN    NaN   0.333353   
3    1980-03-20       NaN        NaN    NaN    NaN    NaN   0.333353   
4    1980-03-25       NaN        NaN    NaN    NaN    NaN   0.312839   
...         ...       ...        ...    ...    ...    ...        ...   
8843 2021-07-26     13.85  20.030001  10.53  15.30  13.45  25.389999   
8844 2021-07-27     13.81  20.120001  10.53  15.20  13.39  25.180000   
8845 2021-07-28     13.81  20.379999  10.52  15.22  13.35  25.180000   
8846 2021-07-29     13.73  19.850000  10.58  15.20  13.33  25.150000   
8847 2021-07-30     13.75  19.969999  10.72  15.11  13.25  24.870001   

                                   ...    Volume                      \
           FDUS    FSK       GAIN  ...      NMFC      OCSL      ORCC   
0           NaN    NaN        NaN  ...       NaN       NaN       NaN   
1           NaN    NaN        NaN  ...       NaN       NaN       NaN   
2           NaN    NaN        NaN  ...       NaN       NaN       NaN   
3           NaN    NaN        NaN  ...       NaN       NaN       NaN   
4           NaN    NaN        NaN  ...       NaN       NaN       NaN   
...         ...    ...        ...  ...       ...       ...       ...   
8843  16.830000  21.09  14.299312  ...  202400.0  423000.0  713200.0   
8844  16.799999  21.00  14.199804  ...  281300.0  274900.0  806200.0   
8845  16.730000  21.01  14.229656  ...  340200.0  525700.0  854500.0   
8846  16.760000  21.02  14.259508  ...  258900.0  327400.0  538700.0   
8847  16.799999  20.99  14.309262  ...  319400.0  443700.0  570200.0   

                                                                             
          PFLT      PNNT       PSEC      SLRC      TCPC      TPVG      TSLX  
0          NaN       NaN        NaN       NaN       NaN       NaN       NaN  
1          NaN       NaN        NaN       NaN       NaN       NaN       NaN  
2          NaN       NaN        NaN       NaN       NaN       NaN       NaN  
3          NaN       NaN        NaN       NaN       NaN       NaN       NaN  
4          NaN       NaN        NaN       NaN       NaN       NaN       NaN  
...        ...       ...        ...       ...       ...       ...       ...  
8843  104300.0  200500.0  1401600.0  122400.0  187100.0   43600.0  182500.0  
8844   83900.0  194900.0  1531100.0   72600.0  108100.0   53100.0  153300.0  
8845  105400.0  101100.0  1008100.0   61600.0  113900.0   86900.0  168600.0  
8846  241900.0  199200.0  1019500.0   79600.0  153300.0   70700.0  239100.0  
8847  181800.0  252900.0  1296100.0   64100.0  148100.0  114500.0  253800.0  

[8848 rows x 151 columns]

### Load dividend history

In [84]:
dividend_history = stock_history[['Date']]
dividend_history

,Date
,
0,1980-03-17
1,1980-03-18
2,1980-03-19
3,1980-03-20
4,1980-03-25
...,...
8843,2021-07-26
8844,2021-07-27
8845,2021-07-28


In [85]:
def findDiv(ticker):
    dividend = yf.Ticker(ticker).dividends
    dividend_df = pd.DataFrame(dividend)
    dividend_tk = dividend_df.rename(columns = {'Dividends' : ticker})
    
    return dividend_tk

In [86]:
for ticker in tickers:
    dividend_df = findDiv(ticker)
    dividend_history = pd.merge(dividend_history, dividend_df, on = 'Date', how = 'left')

/Users/longmini/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [87]:
dividend_history

,Date,"(Date, )",ARCC,FSK,ORCC,HTGC,MAIN,TSLX,GBDC,PSEC,...,CGBD,SLRC,BBDC,CSWC,GAIN,PFLT,TPVG,PNNT,FDUS,GLAD
0,1980-03-17,1980-03-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1980-03-18,1980-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1980-03-19,1980-03-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1980-03-20,1980-03-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1980-03-25,1980-03-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8843,2021-07-26,2021-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8844,2021-07-27,2021-07-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8845,2021-07-28,2021-07-28,NaN,NaN,NaN,NaN,0.205,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8846,2021-07-29,2021-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
dividend_history.fillna(method = 'ffill', inplace = True)
dividend_history.drop(dividend_history.columns[[1]], axis = 1, inplace = True)

In [89]:
dividend_history

,Date,ARCC,FSK,ORCC,HTGC,MAIN,TSLX,GBDC,PSEC,GSBD,...,CGBD,SLRC,BBDC,CSWC,GAIN,PFLT,TPVG,PNNT,FDUS,GLAD
0,1980-03-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1980-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1980-03-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1980-03-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1980-03-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8843,2021-07-26,0.4,0.6,0.31,0.39,0.205,0.41,0.29,0.06,0.45,...,0.36,0.41,0.2,0.53,0.07,0.095,0.36,0.12,0.39,0.065
8844,2021-07-27,0.4,0.6,0.31,0.39,0.205,0.41,0.29,0.06,0.45,...,0.36,0.41,0.2,0.53,0.07,0.095,0.36,0.12,0.39,0.065
8845,2021-07-28,0.4,0.6,0.31,0.39,0.205,0.41,0.29,0.06,0.45,...,0.36,0.41,0.2,0.53,0.07,0.095,0.36,0.12,0.39,0.065
8846,2021-07-29,0.4,0.6,0.31,0.39,0.205,0.41,0.29,0.06,0.45,...,0.36,0.41,0.2,0.53,0.07,0.095,0.36,0.12,0.39,0.065


In [90]:
dividend_history.index = dividend_history['Date']
del dividend_history['Date']

In [91]:
dividend_history

,ARCC,FSK,ORCC,HTGC,MAIN,TSLX,GBDC,PSEC,GSBD,NMFC,...,CGBD,SLRC,BBDC,CSWC,GAIN,PFLT,TPVG,PNNT,FDUS,GLAD
Date,,,,,,,,,,,,,,,,,,,,,
1980-03-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-03-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-26,0.4,0.6,0.31,0.39,0.205,0.41,0.29,0.06,0.45,0.3,...,0.36,0.41,0.2,0.53,0.07,0.095,0.36,0.12,0.39,0.065
2021-07-27,0.4,0.6,0.31,0.39,0.205,0.41,0.29,0.06,0.45,0.3,...,0.36,0.41,0.2,0.53,0.07,0.095,0.36,0.12,0.39,0.065
2021-07-28,0.4,0.6,0.31,0.39,0.205,0.41,0.29,0.06,0.45,0.3,...,0.36,0.41,0.2,0.53,0.07,0.095,0.36,0.12,0.39,0.065


In [92]:
dividend_history.columns = pd.MultiIndex.from_tuples(map(lambda x: ('Dividend', x), dividend_history.columns))

In [93]:
dividend_history

Dividend                                                       ...  \
               ARCC  FSK  ORCC  HTGC   MAIN  TSLX  GBDC  PSEC  GSBD NMFC  ...   
Date                                                                      ...   
1980-03-17      NaN  NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN  NaN  ...   
1980-03-18      NaN  NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN  NaN  ...   
1980-03-19      NaN  NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN  NaN  ...   
1980-03-20      NaN  NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN  NaN  ...   
1980-03-25      NaN  NaN   NaN   NaN    NaN   NaN   NaN   NaN   NaN  NaN  ...   
...             ...  ...   ...   ...    ...   ...   ...   ...   ...  ...  ...   
2021-07-26      0.4  0.6  0.31  0.39  0.205  0.41  0.29  0.06  0.45  0.3  ...   
2021-07-27      0.4  0.6  0.31  0.39  0.205  0.41  0.29  0.06  0.45  0.3  ...   
2021-07-28      0.4  0.6  0.31  0.39  0.205  0.41  0.29  0.06  0.45  0.3  ...   
2021-07-29      0.4  0.6  0.31  0.39  0.205  0.41  0.29  0.06  0.45  0.3  ...   
2021-07-30      0.4  0.6  0.31  0.39  0.205  0.41  0.29  0.06  0.45  0.3  ...   

                                                                         
            CGBD  SLRC BBDC  CSWC  GAIN   PFLT  TPVG  PNNT  FDUS   GLAD  
Date                                                                     
1980-03-17   NaN   NaN  NaN   NaN   NaN    NaN   NaN   NaN   NaN    NaN  
1980-03-18   NaN   NaN  NaN   NaN   NaN    NaN   NaN   NaN   NaN    NaN  
1980-03-19   NaN   NaN  NaN   NaN   NaN    NaN   NaN   NaN   NaN    NaN  
1980-03-20   NaN   NaN  NaN   NaN   NaN    NaN   NaN   NaN   NaN    NaN  
1980-03-25   NaN   NaN  NaN   NaN   NaN    NaN   NaN   NaN   NaN    NaN  
...          ...   ...  ...   ...   ...    ...   ...   ...   ...    ...  
2021-07-26  0.36  0.41  0.2  0.53  0.07  0.095  0.36  0.12  0.39  0.065  
2021-07-27  0.36  0.41  0.2  0.53  0.07  0.095  0.36  0.12  0.39  0.065  
2021-07-28  0.36  0.41  0.2  0.53  0.07  0.095  0.36  0.12  0.39  0.065  
2021-07-29  0.36  0.41  0.2  0.53  0.07  0.095  0.36  0.12  0.39  0.065  
2021-07-30  0.36  0.41  0.2  0.53  0.07  0.095  0.36  0.12  0.39  0.065  

[8848 rows x 25 columns]

In [94]:
stock_history.index = stock_history['Date']
del stock_history['Date']

In [95]:
stock_history

Adj Close                                                        \
                AINV       ARCC   BBDC   BCSF   CGBD       CSWC       FDUS   
Date                                                                         
1980-03-17       NaN        NaN    NaN    NaN    NaN   0.338482        NaN   
1980-03-18       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-19       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-20       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-25       NaN        NaN    NaN    NaN    NaN   0.312839        NaN   
...              ...        ...    ...    ...    ...        ...        ...   
2021-07-26     13.85  20.030001  10.53  15.30  13.45  25.389999  16.830000   
2021-07-27     13.81  20.120001  10.53  15.20  13.39  25.180000  16.799999   
2021-07-28     13.81  20.379999  10.52  15.22  13.35  25.180000  16.730000   
2021-07-29     13.73  19.850000  10.58  15.20  13.33  25.150000  16.760000   
2021-07-30     13.75  19.969999  10.72  15.11  13.25  24.870001  16.799999   

                                     ...    Volume                      \
              FSK       GAIN   GBDC  ...      NMFC      OCSL      ORCC   
Date                                 ...                                 
1980-03-17    NaN        NaN    NaN  ...       NaN       NaN       NaN   
1980-03-18    NaN        NaN    NaN  ...       NaN       NaN       NaN   
1980-03-19    NaN        NaN    NaN  ...       NaN       NaN       NaN   
1980-03-20    NaN        NaN    NaN  ...       NaN       NaN       NaN   
1980-03-25    NaN        NaN    NaN  ...       NaN       NaN       NaN   
...           ...        ...    ...  ...       ...       ...       ...   
2021-07-26  21.09  14.299312  15.94  ...  202400.0  423000.0  713200.0   
2021-07-27  21.00  14.199804  15.95  ...  281300.0  274900.0  806200.0   
2021-07-28  21.01  14.229656  16.00  ...  340200.0  525700.0  854500.0   
2021-07-29  21.02  14.259508  15.87  ...  258900.0  327400.0  538700.0   
2021-07-30  20.99  14.309262  15.78  ...  319400.0  443700.0  570200.0   

                                                                         \
                PFLT      PNNT       PSEC      SLRC      TCPC      TPVG   
Date                                                                      
1980-03-17       NaN       NaN        NaN       NaN       NaN       NaN   
1980-03-18       NaN       NaN        NaN       NaN       NaN       NaN   
1980-03-19       NaN       NaN        NaN       NaN       NaN       NaN   
1980-03-20       NaN       NaN        NaN       NaN       NaN       NaN   
1980-03-25       NaN       NaN        NaN       NaN       NaN       NaN   
...              ...       ...        ...       ...       ...       ...   
2021-07-26  104300.0  200500.0  1401600.0  122400.0  187100.0   43600.0   
2021-07-27   83900.0  194900.0  1531100.0   72600.0  108100.0   53100.0   
2021-07-28  105400.0  101100.0  1008100.0   61600.0  113900.0   86900.0   
2021-07-29  241900.0  199200.0  1019500.0   79600.0  153300.0   70700.0   
2021-07-30  181800.0  252900.0  1296100.0   64100.0  148100.0  114500.0   

                      
                TSLX  
Date                  
1980-03-17       NaN  
1980-03-18       NaN  
1980-03-19       NaN  
1980-03-20       NaN  
1980-03-25       NaN  
...              ...  
2021-07-26  182500.0  
2021-07-27  153300.0  
2021-07-28  168600.0  
2021-07-29  239100.0  
2021-07-30  253800.0  

[8848 rows x 150 columns]

In [97]:
mergedDF = stock_history.join(dividend_history)

In [98]:
mergedDF

Adj Close                                                        \
                AINV       ARCC   BBDC   BCSF   CGBD       CSWC       FDUS   
Date                                                                         
1980-03-17       NaN        NaN    NaN    NaN    NaN   0.338482        NaN   
1980-03-18       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-19       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-20       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-25       NaN        NaN    NaN    NaN    NaN   0.312839        NaN   
...              ...        ...    ...    ...    ...        ...        ...   
2021-07-26     13.85  20.030001  10.53  15.30  13.45  25.389999  16.830000   
2021-07-27     13.81  20.120001  10.53  15.20  13.39  25.180000  16.799999   
2021-07-28     13.81  20.379999  10.52  15.22  13.35  25.180000  16.730000   
2021-07-29     13.73  19.850000  10.58  15.20  13.33  25.150000  16.760000   
2021-07-30     13.75  19.969999  10.72  15.11  13.25  24.870001  16.799999   

                                     ... Dividend                         \
              FSK       GAIN   GBDC  ...     CGBD  SLRC BBDC  CSWC  GAIN   
Date                                 ...                                   
1980-03-17    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-18    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-19    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-20    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-25    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
...           ...        ...    ...  ...      ...   ...  ...   ...   ...   
2021-07-26  21.09  14.299312  15.94  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-27  21.00  14.199804  15.95  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-28  21.01  14.229656  16.00  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-29  21.02  14.259508  15.87  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-30  20.99  14.309262  15.78  ...     0.36  0.41  0.2  0.53  0.07   

                                            
             PFLT  TPVG  PNNT  FDUS   GLAD  
Date                                        
1980-03-17    NaN   NaN   NaN   NaN    NaN  
1980-03-18    NaN   NaN   NaN   NaN    NaN  
1980-03-19    NaN   NaN   NaN   NaN    NaN  
1980-03-20    NaN   NaN   NaN   NaN    NaN  
1980-03-25    NaN   NaN   NaN   NaN    NaN  
...           ...   ...   ...   ...    ...  
2021-07-26  0.095  0.36  0.12  0.39  0.065  
2021-07-27  0.095  0.36  0.12  0.39  0.065  
2021-07-28  0.095  0.36  0.12  0.39  0.065  
2021-07-29  0.095  0.36  0.12  0.39  0.065  
2021-07-30  0.095  0.36  0.12  0.39  0.065  

[8848 rows x 175 columns]

In [104]:
stock_div = mergedDF[['Adj Close', 'Dividend']]

In [107]:
mergedDF.to_csv('stock_div_history.csv')

## 3. Merge two datasets

In [1]:
import pandas as pd

In [5]:
reports = pd.read_csv('financial_reports.csv')
reports

,file_name,document_content,ticker,file_type,period
0,BDC_edgar/sec-edgar-filings/AINV/10-Q/00009478...,10-Q 1 f10q_020605.htm QUARTERLY REPORT Quarte...,AINV,10-Q,05-000318
1,BDC_edgar/sec-edgar-filings/AINV/10-K/00011931...,10-K 1 d10k.htm FORM 10-K Form 10-K Table of C...,AINV,10-K,05-125223
2,BDC_edgar/sec-edgar-filings/AINV/10-Q/00011931...,10-Q 1 d10q.htm FORM 10-Q Form 10-Q Table of C...,AINV,10-Q,05-156270
3,BDC_edgar/sec-edgar-filings/AINV/10-Q/00011931...,10-Q 1 d10q.htm FOR THE QUARTER ENDED SEPTEMBE...,AINV,10-Q,05-215629
4,BDC_edgar/sec-edgar-filings/AINV/10-Q/00011931...,10-Q 1 d10q.htm FORM 10-Q - FOR THE QUARTER EN...,AINV,10-Q,07-022555
...,...,...,...,...,...
1179,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200331.htm 10-Q tslx-10q_202...,TSLX,10-Q,20-021239
1180,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200630.htm 10-Q tslx-10q_202...,TSLX,10-Q,20-036104
1181,BDC_edgar/sec-edgar-filings/TSLX/10-Q/00015645...,10-Q 1 tslx-10q_20200930.htm 10-Q tslx-10q_202...,TSLX,10-Q,20-050467
1182,BDC_edgar/sec-edgar-filings/TSLX/10-K/00015645...,10-K 1 tslx-10k_20201231.htm 10-K tslx-10k_202...,TSLX,10-K,21-006366


In [10]:
stocks = pd.read_csv('stock_div_history.csv', index_col = 0, header = [0,1])
stocks

Adj Close                                                        \
                AINV       ARCC   BBDC   BCSF   CGBD       CSWC       FDUS   
Date                                                                         
1980-03-17       NaN        NaN    NaN    NaN    NaN   0.338482        NaN   
1980-03-18       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-19       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-20       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-25       NaN        NaN    NaN    NaN    NaN   0.312839        NaN   
...              ...        ...    ...    ...    ...        ...        ...   
2021-07-26     13.85  20.030001  10.53  15.30  13.45  25.389999  16.830000   
2021-07-27     13.81  20.120001  10.53  15.20  13.39  25.180000  16.799999   
2021-07-28     13.81  20.379999  10.52  15.22  13.35  25.180000  16.730000   
2021-07-29     13.73  19.850000  10.58  15.20  13.33  25.150000  16.760000   
2021-07-30     13.75  19.969999  10.72  15.11  13.25  24.870001  16.799999   

                                     ... Dividend                         \
              FSK       GAIN   GBDC  ...     CGBD  SLRC BBDC  CSWC  GAIN   
Date                                 ...                                   
1980-03-17    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-18    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-19    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-20    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-25    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
...           ...        ...    ...  ...      ...   ...  ...   ...   ...   
2021-07-26  21.09  14.299312  15.94  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-27  21.00  14.199804  15.95  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-28  21.01  14.229656  16.00  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-29  21.02  14.259508  15.87  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-30  20.99  14.309262  15.78  ...     0.36  0.41  0.2  0.53  0.07   

                                            
             PFLT  TPVG  PNNT  FDUS   GLAD  
Date                                        
1980-03-17    NaN   NaN   NaN   NaN    NaN  
1980-03-18    NaN   NaN   NaN   NaN    NaN  
1980-03-19    NaN   NaN   NaN   NaN    NaN  
1980-03-20    NaN   NaN   NaN   NaN    NaN  
1980-03-25    NaN   NaN   NaN   NaN    NaN  
...           ...   ...   ...   ...    ...  
2021-07-26  0.095  0.36  0.12  0.39  0.065  
2021-07-27  0.095  0.36  0.12  0.39  0.065  
2021-07-28  0.095  0.36  0.12  0.39  0.065  
2021-07-29  0.095  0.36  0.12  0.39  0.065  
2021-07-30  0.095  0.36  0.12  0.39  0.065  

[8848 rows x 175 columns]

In [11]:
stocks_div = stocks[['Adj Close', 'Dividend']]
stocks_div

Adj Close                                                        \
                AINV       ARCC   BBDC   BCSF   CGBD       CSWC       FDUS   
Date                                                                         
1980-03-17       NaN        NaN    NaN    NaN    NaN   0.338482        NaN   
1980-03-18       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-19       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-20       NaN        NaN    NaN    NaN    NaN   0.333353        NaN   
1980-03-25       NaN        NaN    NaN    NaN    NaN   0.312839        NaN   
...              ...        ...    ...    ...    ...        ...        ...   
2021-07-26     13.85  20.030001  10.53  15.30  13.45  25.389999  16.830000   
2021-07-27     13.81  20.120001  10.53  15.20  13.39  25.180000  16.799999   
2021-07-28     13.81  20.379999  10.52  15.22  13.35  25.180000  16.730000   
2021-07-29     13.73  19.850000  10.58  15.20  13.33  25.150000  16.760000   
2021-07-30     13.75  19.969999  10.72  15.11  13.25  24.870001  16.799999   

                                     ... Dividend                         \
              FSK       GAIN   GBDC  ...     CGBD  SLRC BBDC  CSWC  GAIN   
Date                                 ...                                   
1980-03-17    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-18    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-19    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-20    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
1980-03-25    NaN        NaN    NaN  ...      NaN   NaN  NaN   NaN   NaN   
...           ...        ...    ...  ...      ...   ...  ...   ...   ...   
2021-07-26  21.09  14.299312  15.94  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-27  21.00  14.199804  15.95  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-28  21.01  14.229656  16.00  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-29  21.02  14.259508  15.87  ...     0.36  0.41  0.2  0.53  0.07   
2021-07-30  20.99  14.309262  15.78  ...     0.36  0.41  0.2  0.53  0.07   

                                            
             PFLT  TPVG  PNNT  FDUS   GLAD  
Date                                        
1980-03-17    NaN   NaN   NaN   NaN    NaN  
1980-03-18    NaN   NaN   NaN   NaN    NaN  
1980-03-19    NaN   NaN   NaN   NaN    NaN  
1980-03-20    NaN   NaN   NaN   NaN    NaN  
1980-03-25    NaN   NaN   NaN   NaN    NaN  
...           ...   ...   ...   ...    ...  
2021-07-26  0.095  0.36  0.12  0.39  0.065  
2021-07-27  0.095  0.36  0.12  0.39  0.065  
2021-07-28  0.095  0.36  0.12  0.39  0.065  
2021-07-29  0.095  0.36  0.12  0.39  0.065  
2021-07-30  0.095  0.36  0.12  0.39  0.065  

[8848 rows x 50 columns]